In [ ]:
import numpy as np
import colorcet as cc

from bokeh.io import output_file, output_notebook, show
from bokeh.models import LinearColorMapper, WheelZoomTool, ColorBar, Title, Slider, CustomJS
from bokeh.plotting import figure, ColumnDataSource
from bokeh.layouts import column

from vis_tools import read_hysplit_netcdf, grab_gshhg_features, grab_contour_info 

# ignore warnings that arise from my use of NaN's instead of 0's
import warnings
warnings.filterwarnings('ignore', message='invalid value encountered in')
warnings.filterwarnings('ignore', message='No contour levels were found') 
warnings.filterwarnings('ignore', message='Warning: converting a masked element to nan') 

#############################################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '18042918_taupo_15.0_0.01.nc'

# SPECIFY:
ASH_MIN = 10**-2  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
RES = 'i'  # detail of GMT features ('c', 'l', 'i', 'h', 'f')
CONTOURS = 5  # number of contours (or specify None)
#############################################################

model = read_hysplit_netcdf(FILENAME)

src_lon = model.attrs['volcano_location'][1]
src_lat = model.attrs['volcano_location'][0]
lon = model['lon'].values
lat = model['lat'].values

td = model['total_deposition'].values      
td = np.log10(td)  # manually take the log of the ashfall thickness values

# create a list of contour coordinates for Bokeh
if CONTOURS == None:
    contour_stack = [[[],[]] for i in range(len(model['time']))] 
else:
    X, Y = np.meshgrid(model['lon'].values, model['lat'].values)
    contour_stack = []
    for i in range(len(model['time'])):
        lon_all, lat_all = grab_contour_info(X, Y, td[:,:,i], np.linspace(np.log10(ASH_MIN), np.log10(ASH_MAX), CONTOURS))
        contour_stack.append([lon_all, lat_all])         

# transform 3-D array into a list of matrices for Bokeh
image_stack = []
for i in range(len(model['time'])):
    image_stack.append(td[:,:,i])

features = grab_gshhg_features(RES, [1, 2], [166, 180, -48, -34])  # this includes all of NZ

# create Bokeh figure
p = figure(tools='pan, reset')
wz = WheelZoomTool(zoom_on_axis=False)  # restrict zooming behavior
p.add_tools(wz)
p.toolbar.active_scroll = wz

# PLOT model image
image_slice_src = ColumnDataSource(data=dict(x=[np.min(lon)], y=[np.min(lat)], dw=[np.max(lon)-np.min(lon)],\
                                             dh=[np.max(lat)-np.min(lat)], image_slice=[image_stack[0]])
                                  )
cmapper = LinearColorMapper(palette=cc.fire[::-1], low=np.log10(ASH_MIN), high=np.log10(ASH_MAX), nan_color=(0,0,0,0))
p.image('image_slice', x='x', y='y', dw='dw', dh='dh', color_mapper=cmapper, source=image_slice_src)

# PLOT model contours
contour_slice_src = ColumnDataSource(data=dict(xs=contour_stack[0][0], ys=contour_stack[0][1]))
p.multi_line(xs='xs', ys='ys', color='black', line_alpha=0.5, source=contour_slice_src)

# PLOT features
p.multi_line(features['longitude'], features['latitude'], color='black')

# PLOT source location
p.scatter(src_lon, src_lat, size=15, marker='triangle', line_color='black', fill_color='cyan', legend='source')

# add colorbar
color_bar = ColorBar(color_mapper=cmapper, location=(0,0), orientation='horizontal')
color_bar.title = 'log\N{SUBSCRIPT ONE}\N{SUBSCRIPT ZERO} [ ash thickness (mm) ]'
p.add_layout(color_bar, 'below')

p.xaxis.axis_label = 'lon'
p.yaxis.axis_label = 'lat'
p.add_layout(Title(text=FILENAME.split('/')[-1]), 'above')
p.add_layout(Title(text='UNPROJECTED'), 'above')

p.match_aspect = True  # plot degree intervals as equal distances (DISTORTED! BAD! UGLY! BUT ACCURATE!)
p.xgrid.visible = False
p.ygrid.visible = False

# define model time step
time_step = int(np.unique(np.ediff1d(model['time'])).astype('timedelta64[h]').astype(int))  # hours
time_step_src = ColumnDataSource(data=dict(ts=[time_step]))

# create 3D data sources
image_stack_src = ColumnDataSource(data=dict(image_stack=image_stack))
contour_stack_src = ColumnDataSource(data=dict(contour_stack=contour_stack))

# time slider callback function
callback = CustomJS(args=dict(image_slice_src=image_slice_src, image_stack_src=image_stack_src,
                              contour_slice_src=contour_slice_src, contour_stack_src=contour_stack_src,
                              time_step_src=time_step_src), code='''
    var image_stack_src = image_stack_src.data;
    var contour_stack_src = contour_stack_src.data;    
    var time_step_src = time_step_src.data;    
    
    var i = Math.round(hrs.value/time_step_src['ts']);
    
    image_slice_src.data['image_slice'] = [image_stack_src['image_stack'][i]];    
    image_slice_src.change.emit();
        
    contour_slice_src.data['xs'] = contour_stack_src['contour_stack'][i][0]; 
    contour_slice_src.data['ys'] = contour_stack_src['contour_stack'][i][1];    
    contour_slice_src.change.emit();
''')
        
slider = Slider(start=0, end=24, value=0, step=time_step, title='hours after eruption onset', callback=callback)
callback.args['hrs'] = slider
    
interface = column(slider, p)

# UNCOMMENT the line below to show output in the notebook
output_notebook(hide_banner=True); show(interface)

# UNCOMMENT the line below to save the output to a standalone HTML file
#output_file('bokeh_map.html'); show(interface); print('HTML file saved')